In [ ]:
import sys
import os
import json
from shapely.geometry import shape
import numpy as np
import xarray as xr
from pystac_client import Client as StacClient
from dask.distributed import Client, LocalCluster
import stackstac
from typing import Dict
import matplotlib.pyplot as plt
import rioxarray

In [ ]:

cluster = LocalCluster(
    n_workers=3,             # safe for ~12–15 GB RAM
    threads_per_worker=1,    # recommended for raster ops
    memory_limit="4GB"       # 3 workers × 4GB = 12GB total
)
client = Client(cluster)
cluster    # This will display cluster dashboard info


In [ ]:
# Add the folder containing WQI_utils.py
sys.path.append(os.path.abspath("../utils"))

## 2 Import the main functions
from env_utils import environmental_variables
from plots_utils import plot_time_series, plot_seasonal_cycle
from wrapper_plots_utils import workflow_analysis

In [ ]:
bbox = [-82.8, 27.5, -82.2, 28.2]  # Tampa Bay

env_data = environmental_variables(
    bbox=bbox,
    start_year=2019,
    end_year=2024,
    variables=["sst", "precipitation"]
)
# Fetch items per year
items = []
for year in range(start_year, end_year + 1):
    search = client.search(
        collections=[collection],
        bbox=bbox,
        datetime=f"{year}-01-01/{year}-12-31",
        max_items=100
    )
    items.extend(list(search.get_items()))

# Debug: check items
print(f"Number of items found for {var}: {len(items)}")
for i, item in enumerate(items[:5]):
    print(f"Item {i}: geometry={item.geometry}, assets={list(item.assets.keys())}")


In [ ]:
# Add the folder containing WQI_utils.py
sys.path.append(os.path.abspath("../utils"))

## 2 Import the main functions
from env_variables_utils import (
    environmental_variables,
    load_env_timeseries,
    load_env_seasonal,
    plot_env_timeseries,
    plot_env_seasonal
)

# Define bounding box
bbox = [-82.8, 27.5, -82.2, 28.2]  # Tampa Bay

# Step 1: Load time series
ts_data = load_env_timeseries(bbox, variables=["sst", "precipitation"])

# Step 2: Plot time series
plot_env_timeseries(ts_data)

# Step 3: Load seasonal cycle
seasonal_data = load_env_seasonal(bbox, variables=["sst", "precipitation"])

# Step 4: Plot seasonal cycle
plot_env_seasonal(seasonal_data)


In [ ]:
bbox = [-82.8, 27.5, -82.2, 28.2]  # Tampa Bay
env_data = environmental_variables(
    bbox=bbox,
    start_year=2019,
    end_year=2024,
    variables=["sst", "precipitation"]
)

# Access xarray time series
sst_series = env_data["sst"]
precip_series = env_data["precipitation"]

# Quick plot
#sst_series.plot.line(title="SST (Tampa Bay)")
#precip_series.plot.line(title="Precipitation (Tampa Bay)")


In [ ]:
from pystac_client import Client

# Test MODIS SST
stac_url = "https://cmr.earthdata.nasa.gov/stac/LANCEMODIS"
client = Client.open(stac_url)
items = client.search(
    collections=["MODIS_Aqua_L3SM_SST"],
    bbox=bbox,
    datetime="2022-01-01/2022-12-31",
    max_items=10
).get_all_items()

print("Number of SST items:", len(items))
